In [5]:
import pandas as pd
import chardet
import sqlite3

In [ ]:
with open(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Customers_updated.csv", 'rb') as f:
       rawdata = f.read()
       result = chardet.detect(rawdata)
       encoding = result['encoding']

print("Detected Encoding:", encoding)

In [178]:
customers=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Customers_updated.csv",skipinitialspace=True,engine='python')
#data_dictionary=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Data_Dictionary.csv",skipinitialspace=True,engine='python')
products = pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Products_updated.csv",skipinitialspace=True,engine='python')
sales = pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Sales_updated.csv",skipinitialspace=True,engine='python')
stores = pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Stores_updated.csv",skipinitialspace=True,engine='python')
exchange_rates = pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Exchange_Rates_updated.csv",skipinitialspace=True,engine='python')

In [ ]:
customers['Birthday']=pd.to_datetime(customers['Birthday'])
customers.head()

customers['Zip Code'] = pd.to_numeric(customers['Zip Code'], errors='coerce').astype('Int64')  # Use 'Int64' for nullable integer type
print(customers['Zip Code'].dtypes)

has_nan = customers.isna().any().any()

rows_with_nan = customers[customers.isna().any(axis=1)]
print(f"Rows with NaN values:\n{rows_with_nan}")

customers.fillna(method='ffill', inplace=True)

In [154]:
has_nan = data_dictionary.isna().any().any()
has_nan
data_dictionary['Field'] = data_dictionary['Field'].astype(str)
print(data_dictionary['Field'].dtypes)


In [164]:
def clean_and_convert(value):
    if pd.isna(value):
        return None
    # Remove non-numeric characters (e.g., $, spaces)
    cleaned_value = ''.join(filter(str.isdigit, str(value).replace('.', '')))
    if cleaned_value:
        return pd.to_numeric(cleaned_value, errors='coerce')
    return None

has_nan = products.isna().any().any()
has_nan

products['Unit Price USD'] = products['Unit Price USD'].apply(clean_and_convert)
products['Unit Price USD'] = pd.to_numeric(products['Unit Price USD'], errors='coerce').astype('Int64')

print(products['SubcategoryKey'].dtypes)

products.to_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Products_updated.csv", index=False)



In [168]:
sales=pd.read_csv(r"C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Sales.csv",encoding='ascii') 
duplicates_specific = sales[sales.duplicated(subset=['Order Number','CustomerKey','ProductKey'])] 
duplicates_specific 
len(duplicates_specific) 
df_cleaned = sales.drop_duplicates(subset=['Order Number','CustomerKey','StoreKey'])
print(df_cleaned.isna().sum()) 
df_cleaned['Delivery Date'] = pd.to_datetime(df_cleaned['Delivery Date'], errors='coerce') 
df_cleaned['Delivery Date'] = df_cleaned['Delivery Date'].fillna(method='bfill') 
df_cleaned['Order Date'] = pd.to_datetime(df_cleaned['Order Date'], errors='coerce') 
df_cleaned.to_csv(r'C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Sales_updated.csv', index=False)

In [173]:
Exchange_Rates=pd.read_csv(r"C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Exchange_Rates.csv",encoding='ascii') 
duplicates_specific = Exchange_Rates[Exchange_Rates.duplicated(subset=['Date','Currency'])] 
duplicates_specific 
#len(duplicates_specific) 
Exchange_Rates['Date'] = pd.to_datetime(Exchange_Rates['Date'], errors='coerce') 
Exchange_Rates.to_csv(r'C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Exchange_Rates_updated.csv', index=False)

In [ ]:
Stores=pd.read_csv(r"C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Stores.csv",encoding='utf-8') 
print(Stores.isna().sum()) 
Stores['Square Meters'].fillna(Stores['Square Meters'].mean()) 
mean_values = Stores['Square Meters'].min() Stores = Stores.fillna(mean_values) 
mean_values 
duplicates_specific = Stores[Stores.duplicated(subset=['StoreKey'])] 
duplicates_specific 
#len(duplicates_specific) 
Stores['Open Date'] = pd.to_datetime(Stores['Open Date'], errors='coerce') 
Stores.to_csv(r'C:\Users\surej.r\OneDrive - IDP Education Ltd\Desktop\project\Stores_updated.csv', index=False)

In [181]:
merged_data = sales.merge(customers, on='CustomerKey') \
                   .merge(products, on='ProductKey') \
                   .merge(stores, on='StoreKey') \
                   .merge(exchange_rates, left_on=['Currency Code', 'Order Date'], right_on=['Currency', 'Date'])


In [187]:
merged_data=merged_data.drop(columns=['Currency Code','Date'])

In [188]:
merged_data.to_csv(r'C:\Users\surej\OneDrive\Desktop\project\capstone project-2\merged_data.csv', index=False)

In [245]:
conn = sqlite3.connect('DataSpark.db')


In [246]:
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

for i in cursor:
    print(i)

In [ ]:
cursor.execute('''
    CREATE TABLE customers (
        CustomerKey INT PRIMARY KEY,
        Gender VARCHAR,
        Name VARCHAR,
        City VARCHAR,
        State_Code VARCHAR,
        state VARCHAR,
        Zip_Code INT,
        country VARCHAR,
        continent VARCHAR,
        Birthday DATE
    );
    ''')

df=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Customers_updated.csv")

for index, row in df.iterrows():
    sql = '''
        INSERT INTO customers (CustomerKey, Gender, Name,City,State_Code,state,Zip_Code,country,continent,Birthday)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))

In [260]:
cursor.execute('''
    CREATE TABLE Exchange_Rate (
        Date DATE,
        Currency VARCHAR,
        Exchange FLOAT
    );
    ''')

df=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Exchange_Rates_updated.csv")

for index, row in df.iterrows():
    sql = '''
        INSERT INTO Exchange_Rate (Date, Currency, Exchange)
        VALUES (?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))

In [205]:
cursor.execute('''
    CREATE TABLE products (
        ProductKey INT PRIMARY KEY,
        Product_Name VARCHAR,
        Brand VARCHAR,
        Color VARCHAR,
        Unit Cost USD INT,
        Unit Price USD INT,
        SubcategoryKey INT,
        Subcategory VARCHAR,
        CategoryKey INT,
        Category VARCHAR
    );
    ''')

df=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Products_updated.csv")

for index, row in df.iterrows():
    sql = '''
        INSERT INTO products (ProductKey, Product_Name, Brand,Color,Unit_Cost_USD,Unit_Price_USD,SubcategoryKey,Subcategory,CategoryKey,Category)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))

In [214]:
cursor.execute('''
    CREATE TABLE stores (
        StoreKey INT PRIMARY KEY,
        Country VARCHAR,
        State VARCHAR,
        Square_Meter INT,
        Open_Date DATE
    );
    ''')
df=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Stores_updated.csv")

for index, row in df.iterrows():
    sql = '''
        INSERT INTO stores (StoreKey, Country, State, Square_Meter,Open_Date )
        VALUES (?, ?, ?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))

In [219]:
cursor.execute('''
    CREATE TABLE sales (
        Order_Number INT PRIMARY KEY,
        Line_Item INT,
        Order_Date DATE,
        Delivery_Date DATE,
        CustomerKey INT,
        StoreKey INT,
        ProductKey INT,
        Quantity INT,
        Currency_Code VARCHAR,
        FOREIGN KEY (CustomerKey) REFERENCES customers(CustomerKey),
        FOREIGN KEY (StoreKey) REFERENCES stores(StoreKey),
        FOREIGN KEY (ProductKey) REFERENCES products(ProductKey)

    );
    ''')

df=pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\Sales_updated.csv")

for index, row in df.iterrows():
    sql = '''
        INSERT INTO sales (Order_Number, Line_Item, Order_Date,Delivery_Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency_Code)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))

In [228]:
cursor.execute('''
    CREATE TABLE merged_data (
        Order_Number INT PRIMARY KEY,
        Line_Item INT,
        Order_Date DATE,
        Delivery_Date DATE,
        CustomerKey INT,
        StoreKey INT,
        ProductKey INT,
        Quantity INT,
        Gender VARCHAR,
        Name VARCHAR,
        City VARCHAR,
        State_Code VARCHAR,
        state VARCHAR,
        Zip_Code INT,
        country VARCHAR,
        continent VARCHAR,
        Birthday DATE,
        Product_Name VARCHAR,
        Brand VARCHAR,
        Color VARCHAR,
        Unit_Cost_USD INT,
        Unit_Price_USD INT,
        SubcategoryKey INT,
        Subcategory VARCHAR,
        CategoryKey INT,
        Category VARCHAR,
        Country_y VARCHAR,
        State_y VARCHAR,
        Square_Meters INT,
        Open_Date DATE,
        Currency VARHAR,
        Exchange FLOAT

    );
    ''')

df = pd.read_csv(r"C:\Users\surej\OneDrive\Desktop\project\capstone project-2\merged_data.csv")

# Insert each row into the 'merged_data' table
for index, row in df.iterrows():
    sql = '''
        INSERT INTO merged_data (Order_Number, Line_Item, Order_Date, Delivery_Date, CustomerKey, StoreKey, ProductKey, Quantity, Gender, Name, City, State_Code, state, Zip_Code, country, continent, Birthday, Product_Name, Brand, Color, Unit_Cost_USD, Unit_Price_USD, SubcategoryKey, Subcategory, CategoryKey, Category, Country_y, State_y, Square_Meters, Open_Date, Currency, Exchange)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.execute(sql, tuple(row))


In [ ]:
cursor.execute("select * from Exchange_Rate limit 5")
for i in cursor:
    print(i)

In [ ]:
cursor.execute("drop table Exchange_Rates")

In [265]:
conn.commit()